Elongation Demo
================

This example demonstrates how to specify cell shape and contact-inhibited chemotaxis.

Adapated from

    Merks, Roeland MH, et al.
    "Cell elongation is key to in silico replication of in vitro vasculogenesis and subsequent remodeling."
    Developmental biology 289.1 (2006): 44-54.

    Merks, Roeland MH, et al.
    "Contact-inhibited chemotaxis in de novo and sprouting blood-vessel growth."
    PLoS Comput Biol 4.9 (2008): e1000163.

Use the sliders to adjust chemotaxis and elongation during angiogensis

Basic setup
------------

An interactive CC3D simulation can be initialized from a list of core specs.
Start a list of core specs that define the simulation by specifying a two-dimensional simulation
with a 200x200 lattice and second-order Potts neighborhood, and metadata to use multithreading.

In [ ]:
from cc3d.core.PyCoreSpecs import Metadata, PottsCore

# Declare simulation details
dim_x = dim_y = 200
site_len = 2E-6  # Length of a voxel side
step_len = 30.0  # Period of a simulation step

# Declare secretion rate
secr_rate = 1.8E-4 * step_len

specs = [
    Metadata(num_processors=4),
    PottsCore(dim_x=dim_x, dim_y=dim_y)
]

Cell Types
-----------

Define a cell type "T1" and cell type for a wall around the boundary.

In [ ]:
from cc3d.core.PyCoreSpecs import CellTypePlugin

cell_type_specs = CellTypePlugin("T1", "Wall")
cell_type_specs.frozen_set("Wall", True)
specs.append(cell_type_specs)

Volume Constraint
------------------

Assign a volume constraint to all cell types.

In [ ]:
from cc3d.core.PyCoreSpecs import VolumePlugin

volume_specs = VolumePlugin()
volume_specs.param_new("T1", target_volume=50, lambda_volume=2)
specs.append(volume_specs)

Adhesion
---------

Assign uniform adhesion to all cells.

In [ ]:
from cc3d.core.PyCoreSpecs import ContactPlugin

contact_specs = ContactPlugin(neighbor_order=2)
contact_specs.param_new(type_1="Medium", type_2="T1", energy=10)
contact_specs.param_new(type_1="T1", type_2="T1", energy=20)
contact_specs.param_new(type_1="T1", type_2="Wall", energy=50)
specs.append(contact_specs)

Cell Distribution Initialization
---------------------------------

Initialize cells in the domain, and define a steppable that builds a wall during startup.

In [ ]:
from cc3d.core.PyCoreSpecs import UniformInitializer
from cc3d.core.PySteppables import SteppableBasePy

unif_init_specs = UniformInitializer()
unif_init_specs.region_new(pt_min=(7 + 5, 7 + 5, 0), pt_max=(dim_x - 7 - 5, dim_y - 7 - 5, 1),
                           gap=10, width=7, cell_types=["T1"])
specs.append(unif_init_specs)


class ElongationDemoSteppable(SteppableBasePy):

    def start(self):
        """
        Builds a wall at the beginning of simulation
        """
        self.build_wall(self.cell_type.Wall)

steppable = ElongationDemoSteppable()

Chemoattract Diffusion
-----------------------

Apply a PDE field named "F1" solved by DiffusionSolverFE.

In [ ]:
from cc3d.core.PyCoreSpecs import DiffusionSolverFE

diff_solver_specs = DiffusionSolverFE()
f1 = diff_solver_specs.field_new("F1")
f1.diff_data.diff_global = 1E-13 / (site_len * site_len) * step_len
f1.diff_data.decay_types["Medium"] = secr_rate
f1.secretion_data_new("T1", secr_rate)
f1.bcs.x_min_type = "Periodic"
f1.bcs.y_min_type = "Periodic"
specs.append(diff_solver_specs)

Chemotaxis
-----------

Apply contact-mediated chemotaxis to cells by type and field

In [ ]:
from cc3d.core.PyCoreSpecs import ChemotaxisPlugin

chemo_specs = ChemotaxisPlugin()
cs = chemo_specs.param_new(field_name="F1", solver_name="DiffusionSolverFE")
cs.params_new("T1", lambda_chemo=100, towards="Medium")
specs.append(chemo_specs)

Cell Shape
-----------

Apply a length constraint.

In [ ]:
from cc3d.core.PyCoreSpecs import LengthConstraintPlugin

len_specs = LengthConstraintPlugin()
len_specs.params_new("T1", lambda_length=1, target_length=20)
specs.append(len_specs)

Connectivity
-------------

Apply a connectivity constraint to keep cells from fragmenting

In [ ]:
from cc3d.core.PyCoreSpecs import ConnectivityGlobalPlugin

connect_specs = ConnectivityGlobalPlugin(fast=True)
connect_specs.cell_type_append("T1")
specs.append(connect_specs)

Simulation Launch
------------------

Initialize a CC3D simulation service instance and register all simulation specification.

In [ ]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService

cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.register_steppable(steppable=steppable)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

Steering
---------

Add sliders to adjust cell shape and chemotaxis model parameters during simulation execution.

In [ ]:
import ipywidgets

def _cb_length_target(change):
    if change['name'] == 'value':
        len_specs['T1'].target_length = change.new
        len_specs.steer()

def _cb_length_lambda(change):
    if change['name'] == 'value':
        len_specs['T1'].lambda_length = change.new
        len_specs.steer()

def _cb_chemotaxis_lambda(change):
    if change['name'] == 'value':
        chemo_specs['F1']['T1'].lambda_chemo = change.new
        chemo_specs.steer()

slider_length_target = ipywidgets.FloatSlider(
    value=len_specs['T1'].target_length, 
    min=0, 
    max=40, 
    step=1, 
    description='Length constraint target length', 
    continuous_update=False
)
slider_length_lambda = ipywidgets.FloatSlider(
    value=len_specs['T1'].lambda_length, 
    min=0, 
    max=1, 
    step=0.01, 
    description='Length constraint parameter',
    continuous_update=False
)
slider_chemotaxis_lambda = ipywidgets.FloatSlider(
    value=chemo_specs['F1']['T1'].lambda_chemo, 
    min=0, 
    max=1E3, 
    step=1, 
    description='Chemotaxis parameter', 
    continuous_update=False
)

slider_length_target.observe(_cb_length_target, names='value')
slider_length_lambda.observe(_cb_length_lambda, names='value')
slider_chemotaxis_lambda.observe(_cb_chemotaxis_lambda, names='value')

Visualization
--------------

Show a single frame for the cell field and "F1" to visualize simulation data as it is generated 
and all steering widgets.

In [ ]:
from IPython.display import display
from cc3d.core.GraphicsUtils.JupyterGraphicsFrameWidget import CC3DJupyterGraphicsFrameGrid

frame_cells = cc3d_sim.visualize(plot_freq=10)
frame_f1 = cc3d_sim.visualize(plot_freq=10)
frame_f1.field_name = "F1"

frame_grid = CC3DJupyterGraphicsFrameGrid(cols=2)
frame_grid.set_frame(frame_cells, row=0, col=0)
frame_grid.set_frame(frame_f1, row=0, col=1)
frame_grid.control_panel()
frame_grid.show()
display(slider_length_target)
display(slider_length_lambda)
display(slider_chemotaxis_lambda)
display(cc3d_sim.jupyter_run_button())